<h2>Import Modules</h2>

In [41]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import operator
import sys
import plotly
import plotly.graph_objs as go
import plotly.dashboard_objs as dashboard
import plotly.plotly as py


<h2>Read Entire CSV for a day</h2>

In [13]:
data = pd.read_csv("MeasRes_0770_0002.csv")


<h2>Creation of a Dictionary with "Channel number" is a key and its "associated values" is the value</h2>

In [14]:
#load the entire .csv file into a dictionary. 
#while the .csv's rows are times and the columns represent channel numbers,
#it is easier to analyze the changes in the channels if the "channels" dictionary is keyed with the channel numbers.
#For each channel number, its associated value is a list of tuples.  
#In each tuple, tuple[0] is the dB value and tuple[1] is the distance from the primary user threshold.

def distance(dataPoint, threshold):
    return abs(dataPoint + 90) #same as dataPoint - (-90)

#create a dictionary whose keys are the channel numbers and whose values are lists of tuples.
#each tuple contains (dBValue, distance from -90dB)
distances = {}
totalNumberOfRows = 0

for rowNumber, row in data.iterrows():
    channelNumber = 0
    for value in row:
        distanceFromMinus90 = distance(value, -90)
        associatedValue = (value, distanceFromMinus90)
        if channelNumber not in distances:
            distances[channelNumber] = [associatedValue]
        else:
            distances[channelNumber].append(associatedValue)
        channelNumber+=1

numChannels = len(distances)
numDataPointsPerChannel = len(distances[0]) #This is true since each row has the same number of columns

<h2>Graph function to plot the scatter plot using plotly from the dictionary</h2>

In [25]:
def graph(distances, lowerChannelNo, upperChannelNo, maxDistanceFromThreshold, path):
    
    #create 3 parallel lists to pass to the plotly scatter plot.
    channels = []
    dBValues = []
    relativeDistances= []

    for channelNo in range(lowerChannelNo, upperChannelNo+1):
            for pair in distances[channelNo]:
                if pair[1] <=maxDistanceFromThreshold:
                    channels.append(channelNo)
                    #dBValues.append(pair[0])  #pair[0] is the dB value
                    relativeDistances.append(pair[1]) #pair[1] is the distance
                    
    
    #create a 4th list        
    sizes = list(map(lambda distance: 10-distance, relativeDistances))

    allValues = go.Scatter(
        x = channels,
        y = relativeDistances,
        mode='markers',
        marker=dict(
            size=sizes,
            color = "red", #set color equal to a variable
        )
    )
    data = [allValues]

    plotly.offline.plot(data, filename=path)


In [52]:
graph(distances, 100, 200, 10, "channelData.html")

<h2>Breaking down a entire day CSV in quater length of a day</h2>

In [4]:
def graphProbabilities(day, part, distances, lowerChannelNo, upperChannelNo, maxDistanceFromThreshold):
    channels = []
    probabilities = []
    colors = []
    sizes = []
    
    #for each channel, calculate the probability of vulnerability w.r.t. the threshold.
    for channelNo in range(lowerChannelNo, upperChannelNo+1):
        totalVulnerable = 0
        for pair in distances[channelNo]:
            if pair[1]<=maxDistanceFromThreshold:
                totalVulnerable+=1
        channels.append(channelNo)
        probabilities.append(totalVulnerable / len(distances[channelNo]))
        
    #map the color function over the probabilities to obtain a list of colors to pass to scatter()
    colors = list(map(setColor, probabilities))
   
    #map the size function over the probabilities to obtain a list of sizes to pass to scatter()
    sizes = list(map(setSize, probabilities))
    
    graph = [go.Scatter(x = channels, y=probabilities, mode="markers", 
                       marker = dict(color = colors, size = sizes))]
    
    #calculate the start and end times for each file in order to use them as a title for the plot
    start, startingPartOfDay = startTime(part)
    end, endingPartOfDay = endTime(part)
    layout = go.Layout(title="Vulnerability analysis of spectrum -- Day " + str(day) +": " 
                       + str(normalizeTime(start)) +startingPartOfDay+ " - " + str(normalizeTime(end)) 
                       + endingPartOfDay,
                       xaxis = dict(title="Channel Number"), 
                       yaxis=dict(title="probability of vulnerability (w/distance = " + str(maxDistanceFromThreshold)
                                 +"dB)"))
    figure = go.Figure(data=graph, layout=layout)
    return py.plot(figure, filename="Day"+str(day)+"Part"+str(part)+"Distance"+str(maxDistanceFromThreshold)
                        +".html", auto_open=False)
 

def startTime(part):
    result = 11 + (part-1)*6
    if result > 24:
        result-=24
    timeOfDay=None
    if result > 12:
        timeOfDay = "PM"
    else:
        timeOfDay = "AM"
    return result, timeOfDay

def endTime(part):
    result, time = startTime(part)
    result+=6
    if result>24:
        result-=24
    timeOfDay=None
    if result>12:
        timeOfDay="PM"
    else:
        timeOfDay="AM"
    return result, timeOfDay

def normalizeTime(time):
    if time>12:
        time-=12
    return time

#make particularly vulnerable channels appear as larger bubbles in the scatter plot
def setSize(probability):
    if probability>=0.9:
        return 10
    else:
        return 7

#set the color to green (not so vulnerable), blue(around 50% probability)
#yellow(75%) or red(90% probability of higher)
def setColor(probability):
    if probability <=0.25:
        return "rgb(27, 235, 27)"
    elif probability <=0.5:
        return "rgb(27, 48, 235)"
    elif probability <=0.75: 
        return "rgb(255, 215, 0)"
    else: 
        return "rgb(255, 0, 0)"

In [81]:
graphProbabilities(1, 2, distances, 1, len(distances)-1, 10)